In [1]:
breadboard_repo_path = r'C:\Users\Alex\Dropbox (MIT)\lab side projects\control software\breadboard-python-client\\'
import sys
sys.path.insert(0, breadboard_repo_path)
from breadboard import BreadboardClient
bc = BreadboardClient(config_path='API_CONFIG_fermi1.json') # enter your path to the API_config
from IPython.display import Image
import ipywidgets as widgets
import qgrid
import pandas as pd
import time
import warnings
from measurement_directory import *
from numpy import nan
from ipywidgets import interact
import os
from image_watchdog import getFileList

def get_newest_df(watchfolder, optional_column_names = []):
    files, _ = getFileList(watchfolder)
    run_ids = [run_id_from_filename(file) for file in files]
    return bc.get_runs_df_from_ids(run_ids, optional_column_names = optional_column_names)

print('today\'s measurement(s):')
print(todays_measurements())
watchfolder = measurement_directory()

## Run the next two cells to load the log from one of today's runs.

In [2]:
def edit_image_log(new_column = False, extra_display_vars = []):
    df = get_newest_df(watchfolder, optional_column_names = ['editable_variables'] + 
                       extra_display_vars)
    
    if 'editable_variables' in df.columns:
        editable_vars = df.loc[0,'editable_vars']
    else:
        editable_vars = []
    
#     if new_column:
#         column_name = input('enter new column name (use CamelCase): ')
#         if column_name not in df.columns:
#             df[column_name] = nan
#             editable_vars += [column_name]
#             df['editable_variables'] = editable_vars
#         else:
#             warnings.warn('column \'' + column_name + '\' already exists')
    
    col_opts = { 'editable': False }
    col_defs = {}
    for col_name in editable_vars + ['badshot', 'notes']:
        col_defs[col_name] = { 'editable': True }
    col_defs['badshot']['ColumnWidth'] = 50
    qgrid_widget = qgrid.show_grid(df, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 100},
                                  column_options=col_opts, column_definitions=col_defs)
    return qgrid_widget

def save_image_log(event, qgrid_widget):
    row_idx = int(event['new'][0])
    df = qgrid_widget.get_changed_df()
    df_updated_row = df.loc[row_idx]
    updated_row_dict = {}
    for key in df_updated_row.index:
        if key == 'notes':
            continue
        elif key == 'badshot':
            updated_row_dict[key] = bool(df_updated_row.loc[key])
        else:
            updated_row_dict[key] = df_updated_row.loc[key]
    run_id = df_updated_row['run_id']
    run_dict = bc._send_message(
            'get', '/runs/' + str(run_id) + '/').json()
    run_dict['notes'] = df_updated_row['notes']
    run_dict['parameters'].update(updated_row_dict)
    payload = json.dumps(run_dict)
    bc._send_message('put', '/runs/' + str(run_id) + '/', data=payload)
    qgrid_widget.close()
    qgrid_widget = edit_image_log()
    qgrid_widget.on('cell_edited',save_image_log)
    display(qgrid_widget)

# image_viewer = widgets.Output(layout={'border': '1px solid black'})
# def show_selected_image(event, qgrid_widget):
#     image_viewer.clear_output()
#     row_idx = int(event['new'][0])
#     df = pd.read_csv(filename)
#     raw_image_filename = df.loc[row_idx, 'filename0']
#     filepath = measurement_dir + '\\' + raw_image_filename.replace('.spe', '.jpg')
#     with image_viewer:
#         if os.path.exists(filepath) and '.jpg' in filepath:
#             pil_img = Image(filepath)
#             display(pil_img)
#         else:
#             display('no jpg preview at ' + filepath)


# def add_new_column(new_column_in = False):
#     global new_column
#     new_column = new_column_in

# mode_select_widget = interact(add_new_column,new_column_in=[False, True])

interactive(children=(Dropdown(description='new_column_in', options=(False, True), value=False), Output()), _d…

In [5]:
%matplotlib notebook
global filename
measurement_dir = measurement_directory()
filename = measurement_dir + r'\image_log.csv'
qgrid_widget = edit_image_log(new_column)
qgrid_widget.on('cell_edited',save_image_log)
qgrid_widget.on('selection_changed', show_selected_image)
display(qgrid_widget)
display(image_viewer)

Enter name for this set of runs:foo


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

Output(layout=Layout(border='1px solid black'))

In [24]:
display('no image')

'no image'

In [20]:
df

NameError: name 'df' is not defined